Note: must turn off radar in settings before recording

In [6]:
from google.colab import drive 
drive.mount('/content/drive')

#imports + file upload
print("need to prune imports")
#h+=1

%tensorflow_version 2.x

import os

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Sequential
from keras.layers import Dense,Activation,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization

import time
import random
import numpy as np
from tqdm import tqdm
from PIL import Image

assert len(tf.config.list_physical_devices('GPU')) > 0

from google.colab import files
from skimage.color import rgb2gray
from collections import Counter
from random import shuffle
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt 
#uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
need to prune imports


In [7]:
#data processing

#next steps: use GAN to debias dataset (MIT 6.S191 lec 4, lab 2)

def data_processing(start_val,num_files):
  s_turn = []
  m_turn = []
  adj_turn = []
  no_turn = []
  balanced_data = []
  starting_value = start_val
  continue_loop = True
  #start_time = time.time()
  while continue_loop:
    data_file_name = '/content/drive/MyDrive/GTA Driving Data/processed-training_data-{}.npy'.format(starting_value)
    if os.path.isfile(data_file_name) and starting_value < start_val + num_files:
      print(starting_value)
      starting_value += 1
      processed_data = np.load(data_file_name,allow_pickle=True)    
      #print(len(processed_data))
      #h+= 1
      #i = 0
      for entry in processed_data:
        #print(i)
        #i += 1
        #np_entry =print(entry[0])
        y = entry[0][0:3]
        y_ls = y[2]
        x= entry[1]
        #x = rgb2gray(entry[1])
        #flipping images
        y_flipped = [y[0],y[1],1-y_ls]
        x_flipped = cv2.flip(x, 1)
        #print(y_ls)
        if (y_ls > 0.875 or y_ls < 0.125):
          #very sharp turn, but it doesn't warrant its own array
          s_turn.append([x,y])
          #s_turn.append([x_flipped,y_flipped])
        elif (y_ls > 0.8 or y_ls < 0.2):
          #sharp turn
          #s_turn.append([x_flipped,y_flipped])
          s_turn.append([x,y])
        elif (y_ls > 0.7 or y_ls < 0.3):
          #medium turn
          m_turn.append([x,y])
          #m_turn.append([x_flipped,y_flipped])

        elif (y_ls > 0.65 or y_ls < 0.35):
          #light turn, doesn't warrant its own array
          m_turn.append([x,y])
          #m_turn.append([x_flipped,y_flipped])

        elif (y_ls > 0.575 or y_ls < 0.425):
          #adjustments
          adj_turn.append([x,y])
          #adj_turn.append([x_flipped,y_flipped])
        else:
          #no turn
          no_turn.append([x,y])
          #no_turn.append([x_flipped,y_flipped])
    else:
      if not (starting_value < start_val + num_files):
        continue_loop = False
      else:
        starting_value += 1

  shuffle(no_turn)
  #without shuffling, array adjustment in next step would mean that the training set is full of mirror repeats 
  #we would prefer to have [img12_mirrored,img1] rather than [img1_mirrored, img1]
  #shuffling allows for this

  balanced_data = s_turn + m_turn + adj_turn + no_turn[0:len(adj_turn)]
  #we want agent to make small adjustments, so adj_turn isn't being adjusted
  #we don't want agent to just go straight, so less of no_turn is being added
  

  """
  arr_lengths = [len(adj_turn),len(adj_turn),len(m_turn),len(s_turn)]
  arr_names = ["no turn (trimmed)","adjustment turn","medium turn", "sharp turn"]
  
  plt.bar(arr_names,arr_lengths)
  plt.show()
  """

  no_turn = []
  s_turn = []
  m_turn = []
  adj_turn = []
  shuffle(balanced_data)
  X = []
  Y = []
  for entry in balanced_data:
    entry[0] = rgb2gray(entry[0])


  X , Y = zip(*balanced_data)

  print("done! have {} test cases".format(len(balanced_data)))
  #print("time to run time: {}".format(time.time()-start_time))
  balanced_data = []

  X_train_3dim = tf.convert_to_tensor(X)
  new_shape = list(X_train_3dim.shape) + [1]
  X_train = tf.reshape(X_train_3dim,new_shape)

  Y_train_3dim = tf.convert_to_tensor(Y)
  new_shape = list(Y_train_3dim.shape) + [1]
  Y_train = tf.transpose(tf.reshape(Y_train_3dim,new_shape),perm = [0,2,1])
  
  return X_train,Y_train

In [8]:
#saving chunk of training data as testing data (now irrelevant)

#PERCENT_TEST = 20

#test_index_start = int(PERCENT_TEST*len(balanced_data)/100)
#train_data = balanced_data[:-test_index_start]
#test_data = balanced_data[-test_index_start:]

In [ ]:

"""
import mxnet as mx
from mxnet import np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
"""

#set up alexnet


def build_alexnet_model():
  #image_shape = (80,60,1)
  np.random.seed(1000)
  #https://www.tensorflow.org/api_docs/python/tf/keras/layers/InputLayer
  model = tf.keras.Sequential([
    #tf.keras.layers.InputLayer(input_shape=image_shape),
    tf.keras.layers.Conv2D(filters=96, kernel_size=(5,5),strides=(2,2),activation=tf.nn.relu), 
    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),activation=tf.nn.relu), 
    #tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)),
    #tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),activation=tf.nn.relu), 
    tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),activation=tf.nn.relu), 
    tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),activation=tf.nn.relu), 
    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(4096, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation=tf.keras.activations.linear)])
  
  return model

def build_cnn_model():
      cnn_model = tf.keras.Sequential([
        # Here, we use a larger 11 x 11 window to capture objects. At the same
        # time, we use a stride of 2 to reduce the height and width of
        # the output.
        tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=2),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        #tf.keras.layers.BatchNormalization(axis=3),
        # Make the convolution window smaller, set padding to 2 for consistent
        # height and width across the input and output, and increase the
        # number of output channels
        tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same',
                               activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        #tf.keras.layers.BatchNormalization(),
        # Use three successive convolutional layers and a smaller convolution
        # window. Except for the final convolutional layer, the number of
        # output channels is further increased. Pooling layers are not used to
        # reduce the height and width of input after the first two
        # convolutional layers
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                               activation='relu'),
        #tf.keras.layers.BatchNormalization(axis=-1),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                               activation='relu'),
        #tf.keras.layers.BatchNormalization(axis=-1),
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',
                               activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        #tf.keras.layers.BatchNormalization(axis=-1),
        tf.keras.layers.Flatten(),
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Output layer. Because control vector is made up of 3 values, 
        # this dense layer is made up of only 3 units
        tf.keras.layers.Dense(3)])
      
      return cnn_model


#x = np.random.rand(2,160,120,1)
#X = tf.convert_to_tensor(x)

# Initialize the model by passing some data through
#predictions = cnn_model.predict(X_train_0)
#print(predictions)
#print(predictions.shape)
# Print the summary of the layers in the model.
#print(cnn_model.summary())

#load cnn model, load arrays
time_vals = []
train_accuracy = []
test_accuracy = []

epoch_to_load = int(input("which epoch to load?"))
if epoch_to_load > 0:
  cnn_model = load_weights('/content/drive/MyDrive/GTA Driving Data/training weights/gta_cnn_model_epoch_{}.h5'.format(epoch_to_load))
  graph_data = np.load('/content/drive/MyDrive/GTA Driving Data/progress data/graph_data.npy',allow_pickle=True)
  time_vals = graph_data[0][0:epoch_to_load]
  train_accuracy = graph_data[1][0:epoch_to_load]
  test_accuracy = graph_data[2][0:epoch_to_load]

else:
  cnn_model = build_cnn_model()
  cnn_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1), 
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['accuracy'])

BATCH_SIZE = 32
EPOCHS = 5

files_in_epoch = 0
accuracy_store = 0
start_val = 0
num_files_to_retrieve = 10
num_files_to_test = 200
for epoch in range(epoch_to_load+1,EPOCHS+1):
  print('-------------STARTING EPOCH {}-------------'.format(epoch))
  files_in_epoch = 0
  accuracy_store = 0
  while start_val < num_files_to_test-num_files_to_retrieve*3:
    if(start_val < 83):
      X_train, Y_train = data_processing(start_val,num_files_to_retrieve)
      #print("len {}".format(len(X_train)))
      start_val += num_files_to_retrieve
    else:
      X_train, Y_train = data_processing(start_val,num_files_to_retrieve*3)
      start_val += num_files_to_retrieve*3
    
    num_files = X_train.shape[0]
    files_in_epoch += num_files

    history = cnn_model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=1)#,callbacks=[cp_callback])
    #weights accuracy by number of files it is measured over
    group_accuracy = num_files*history.history['accuracy'][0]
    accuracy_store += group_accuracy  

  #train on tail end of dataset
  X_train, Y_train = data_processing(start_val,num_files_to_test-start_val+1)

  num_files = X_train.shape[0]
  files_in_epoch += num_files

  history = cnn_model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=1)#,callbacks=[cp_callback])
  
  group_accuracy = num_files*history.history['accuracy'][0]
  accuracy_store += group_accuracy

  #appends average epoch accuracy
  train_accuracy.append(accuracy_store/(files_in_epoch))
  time_vals.append(epoch)

  X_test, Y_test = data_processing(201,33)
  test_loss, test_acc = cnn_model.evaluate(X_test,Y_test)
  print('Test accuracy:', test_acc)
  test_accuracy.append(test_acc)

  #saving model
  if epoch % 5 == 0:
    print("SAVED")
    file_name = "/content/drive/MyDrive/GTA Driving Data/training weights/gta_cnn_model_epoch_{}.h5".format(epoch)
    cnn_model.save(file_name)
    graph_data = [time_vals,train_accuracy,test_accuracy]
    np.save('/content/drive/MyDrive/GTA Driving Data/progress data/graph_data.npy',graph_data)
  
  start_val = 0

plt.plot(time_vals, test_accuracy, label = "test accuracy")
plt.plot(time_vals, train_accuracy, label = "train accuracy")
plt.show()

In [ ]:
"""
#defining lstm model
#rnn is later
def build_LSTM_model():#vocab_size, embedding_dim, rnn_units, batch_size):
  model = keras.Sequential(
      [
       layers.ConvLSTM2D(
           filters=40,kernel_size=(3,3),padding='same',return_sequences=True
       ),
       layers.BatchNormalization(),
       layers.ConvLSTM2D(
           filters=40,kernel_size=(3,3),padding='same',return_sequences=True
       ),
       layers.BatchNormalization()
       ])
  model.compile(loss=keras.losses.MeanSquaredError(),optimizer='adam',metrics=["accuracy"])
  return model

x = np.random.rand(1600,120,160,1)
X = tf.convert_to_tensor(x)
print(X.shape[0])


rnn_model = build_LSTM_model()
# Initialize the model by passing some data through
predictions = rnn_model.predict(X)

#print(predictions)
# Print the summary of the layers in the model.
print(rnn_model.summary())



    # Layer 1: Embedding layer to transform indices into dense vectors 
    #   of a fixed embedding size
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units. 
    # TODO: Call the LSTM function defined above to add this layer.
    LSTM(rnn_units),

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size. 
    # TODO: Add the Dense layer.
    tf.keras.layers.Dense(vocab_size)
  ])
"""

  #return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
#model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)

In [ ]:
#figure out batches w/ input data, set that up, break them down into 100 frames

In [ ]:
"""
# saving and loading the model weights
 
# save model
model.save_weights('gfgModelWeights')
print('Model Saved!')
 
# load model
savedModel = model.load_weights('gfgModelWeights')
print('Model Loaded!')
"""